In [ ]:
text = "The Tokenizer is a necessary and pervasive component of Large Language Models (LLMs), where it translates between strings and tokens (text chunks). Tokenizers are a completely separate stage of the LLM pipeline: they have their own training sets, training algorithms (Byte Pair Encoding), and after training implement two fundamental functions: encode() from strings to tokens, and decode() back from tokens to strings. In this lecture we build from scratch the Tokenizer used in the GPT series from OpenAI. In the process, we will see that a lot of weird behaviors and problems of LLMs actually trace back to tokenization. We'll go through a number of these issues, discuss why tokenization is at fault, and why someone out there ideally finds a way to delete this stage entirely."
tokens = text.encode('utf-8')
tokens = list(map(int, tokens))

print('____')
print(text)
print('length', len(text))
print('___')
print(tokens)
print('length', len(tokens))

____
The Tokenizer is a necessary and pervasive component of Large Language Models (LLMs), where it translates between strings and tokens (text chunks). Tokenizers are a completely separate stage of the LLM pipeline: they have their own training sets, training algorithms (Byte Pair Encoding), and after training implement two fundamental functions: encode() from strings to tokens, and decode() back from tokens to strings. In this lecture we build from scratch the Tokenizer used in the GPT series from OpenAI. In the process, we will see that a lot of weird behaviors and problems of LLMs actually trace back to tokenization. We'll go through a number of these issues, discuss why tokenization is at fault, and why someone out there ideally finds a way to delete this stage entirely.
length 781
___
[84, 104, 101, 32, 84, 111, 107, 101, 110, 105, 122, 101, 114, 32, 105, 115, 32, 97, 32, 110, 101, 99, 101, 115, 115, 97, 114, 121, 32, 97, 110, 100, 32, 112, 101, 114, 118, 97, 115, 105, 118, 101, 

In [ ]:
chr(84),chr(104)

('T', 'h')

In [ ]:
def get_stats(ids):
    counts = {}
    for pair in  zip(ids, ids[1:]):
        counts[pair] = counts.get(pair,0) + 1
    return counts

stats  = get_stats(tokens)
print(stats)

##print(sorted(((v,k) for k, v in stats.items()), reverse=True))

{(84, 104): 1, (104, 101): 10, (101, 32): 24, (32, 84): 3, (84, 111): 3, (111, 107): 8, (107, 101): 8, (101, 110): 15, (110, 105): 8, (105, 122): 5, (122, 101): 3, (101, 114): 9, (114, 32): 6, (32, 105): 7, (105, 115): 6, (115, 32): 18, (32, 97): 16, (97, 32): 5, (32, 110): 2, (110, 101): 4, (101, 99): 3, (99, 101): 3, (101, 115): 6, (115, 115): 4, (115, 97): 1, (97, 114): 4, (114, 121): 1, (121, 32): 8, (97, 110): 8, (110, 100): 8, (100, 32): 9, (32, 112): 4, (112, 101): 3, (114, 118): 1, (118, 97): 1, (97, 115): 1, (115, 105): 1, (105, 118): 1, (118, 101): 2, (32, 99): 3, (99, 111): 5, (111, 109): 7, (109, 112): 3, (112, 111): 1, (111, 110): 5, (110, 116): 4, (116, 32): 8, (32, 111): 7, (111, 102): 5, (102, 32): 5, (32, 76): 4, (76, 97): 2, (114, 103): 1, (103, 101): 4, (110, 103): 8, (103, 117): 1, (117, 97): 2, (97, 103): 3, (32, 77): 1, (77, 111): 1, (111, 100): 4, (100, 101): 6, (101, 108): 5, (108, 115): 1, (32, 40): 3, (40, 76): 1, (76, 76): 3, (76, 77): 3, (77, 115): 2, (115, 

In [ ]:
top_pair = max(stats, key=stats.get)
top_pair

(32, 116)

In [ ]:
def merge(ids, pair, idx):
  newids = []
  i=0
  while i < len(ids):
    if i<len(ids)-1 and ids[i]== pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i+=2
    else:
      newids.append(ids[i])
      i+=1
  return newids

print(merge([5,6,6,7,9,1], (6,7), 99))
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print('length', len(tokens2))

[5, 6, 99, 9, 1]
[84, 104, 101, 32, 84, 111, 107, 101, 110, 105, 122, 101, 114, 32, 105, 115, 32, 97, 32, 110, 101, 99, 101, 115, 115, 97, 114, 121, 32, 97, 110, 100, 32, 112, 101, 114, 118, 97, 115, 105, 118, 101, 32, 99, 111, 109, 112, 111, 110, 101, 110, 116, 32, 111, 102, 32, 76, 97, 114, 103, 101, 32, 76, 97, 110, 103, 117, 97, 103, 101, 32, 77, 111, 100, 101, 108, 115, 32, 40, 76, 76, 77, 115, 41, 44, 32, 119, 104, 101, 114, 101, 32, 105, 116, 256, 114, 97, 110, 115, 108, 97, 116, 101, 115, 32, 98, 101, 116, 119, 101, 101, 110, 32, 115, 116, 114, 105, 110, 103, 115, 32, 97, 110, 100, 256, 111, 107, 101, 110, 115, 32, 40, 116, 101, 120, 116, 32, 99, 104, 117, 110, 107, 115, 41, 46, 32, 84, 111, 107, 101, 110, 105, 122, 101, 114, 115, 32, 97, 114, 101, 32, 97, 32, 99, 111, 109, 112, 108, 101, 116, 101, 108, 121, 32, 115, 101, 112, 97, 114, 97, 116, 101, 32, 115, 116, 97, 103, 101, 32, 111, 102, 256, 104, 101, 32, 76, 76, 77, 32, 112, 105, 112, 101, 108, 105, 110, 101, 58, 256, 104,

In [ ]:

vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"merging {pair} into a new token {idx}")
    ids = merge(ids,pair,idx)
    merges[pair] = idx




merging (32, 116) into a new token 256
merging (101, 32) into a new token 257
merging (115, 32) into a new token 258
merging (101, 110) into a new token 259
merging (105, 110) into a new token 260
merging (256, 104) into a new token 261
merging (101, 114) into a new token 262
merging (32, 115) into a new token 263
merging (256, 111) into a new token 264
merging (107, 259) into a new token 265
merging (32, 97) into a new token 266
merging (258, 97) into a new token 267
merging (110, 100) into a new token 268
merging (111, 109) into a new token 269
merging (114, 97) into a new token 270
merging (260, 103) into a new token 271
merging (32, 102) into a new token 272
merging (100, 101) into a new token 273
merging (265, 105) into a new token 274
merging (274, 122) into a new token 275


In [ ]:
print("tokens length:", len(tokens))
print("ids length", len(ids ))
print(f"compression ratio: {len(tokens)/ len(ids):.2f}X")

tokens length: 781
ids length 576
compression ratio: 1.36X


In [ ]:
##decode

In [ ]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1) , idx in merges.items():
  vocab[idx] = vocab[p0]  + vocab[p1]

def decode(ids):
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode('latin-1')


  return text

print(decode([201]))

É


In [ ]:
def encode(text):

  tokens = list(text.encode('utf-8'))
  while len(tokens) >= 2:
    stats= get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float('inf')))
    if pair not in merges:
      break

      idx = merges[pair]
      tokens = merge(tokens, pair, idx)
  return tokens

print(encode(""))

[]


In [ ]:
print(decode(encode('hello world')))

hello world


In [ ]:
text2 = decode(encode(text))
print(text2 == text)